In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: C:\D disk\ID2223_lab1\mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-05 11:31:38,685 INFO: Initializing external client
2025-11-05 11:31:38,687 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-05 11:31:43,161 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1271980


'{"country": "china", "city": "beijing", "street": "dongchengdongsi", "aqicn_url": "https://api.waqi.info/feed/@446", "latitude": "39.929", "longitude": "116.417"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,192.0,china,beijing,dongchengdongsi,2025-11-05,https://api.waqi.info/feed/@446


In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [7]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 40.0°N 116.5°E
Elevation 48.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-05,9.90,0.0,1.800000,360.000000,beijing
1,2025-11-06,9.55,0.0,3.054701,315.000092,beijing
2,2025-11-07,7.70,0.7,6.130579,3.366400,beijing
3,2025-11-08,9.15,0.0,3.976330,264.805664,beijing
4,2025-11-09,6.70,0.0,6.297428,239.036316,beijing
5,2025-11-10,7.10,0.0,5.959060,205.016861,beijing
6,2025-11-11,8.90,0.0,8.145870,224.999893,beijing


In [8]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [9]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-05 11:32:27,376 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1271980/fs/1265734/fg/1637760


Uploading Dataframe: 100.00% |█████████████████████████████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1271980/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 735264
         }
       },
       "result": {
         "observed_value": 192.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-05T03:32:27.000375Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expecta

In [16]:
air_quality_fg.read().sort_values("date", ascending=False).head(10)   # 看最近 10 天

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.57s) 


,date,pm25,country,city,street,url
4243,2025-11-05 00:00:00+00:00,192.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
4242,2025-11-04 00:00:00+00:00,157.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/beijing/dongchengdo...
3972,2025-11-03 00:00:00+00:00,93.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
4042,2025-11-02 00:00:00+00:00,45.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
256,2025-11-01 00:00:00+00:00,115.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
2104,2025-10-31 00:00:00+00:00,95.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
1043,2025-10-30 00:00:00+00:00,178.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
3924,2025-10-29 00:00:00+00:00,198.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
914,2025-10-28 00:00:00+00:00,144.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446
3804,2025-10-27 00:00:00+00:00,65.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446


In [18]:
# FOR C GRADE

# ===== Fix: compute lags using strictly previous days from v1, then insert today's row into v2 =====
air_quality_fg_v2 = fs.get_feature_group(name='air_quality', version=2)

# 今天这行（补齐主键与必备列）
today_core = aq_today_df[['date', 'pm25']].copy()
today_core['pm25'] = today_core['pm25'].astype('float32')
today_core['country'] = country
today_core['city'] = city
today_core['street'] = street
today_core['url'] = aqicn_url

# 统一时间为无时区
today_core['date'] = pd.to_datetime(today_core['date'], utc=True).dt.tz_convert(None)
today_ts = pd.to_datetime(today)  # 无时区

# 仅用“今天之前”的历史作为 lag 基准
hist_v1 = air_quality_fg.read()[['date','country','city','street','pm25']].copy()
hist_v1['date'] = pd.to_datetime(hist_v1['date'], utc=True).dt.tz_convert(None)
hist_v1 = hist_v1[
    (hist_v1['country'] == country) &
    (hist_v1['city'] == city) &
    (hist_v1['street'] == street) &
    (hist_v1['date'] < today_ts)           # ★ 关键：严格小于今天
].sort_values('date')

# 如果有同一天多条，保留最后一条
hist_v1 = hist_v1.drop_duplicates(subset=['date'], keep='last')

# 取最后 3 天历史，直接赋值给 lag1/2/3
last3 = hist_v1.tail(3)['pm25'].tolist()

if len(last3) == 3:
    today_row = today_core.copy()
    today_row['pm25_lag1'] = last3[-1]
    today_row['pm25_lag2'] = last3[-2]
    today_row['pm25_lag3'] = last3[-3]

    # ★ 关键：统一数值类型为 float32，与 FG schema 对齐
    today_row[['pm25', 'pm25_lag1', 'pm25_lag2', 'pm25_lag3']] = \
        today_row[['pm25', 'pm25_lag1', 'pm25_lag2', 'pm25_lag3']].astype('float32')

    # 按 v2 schema 补齐并排序列
    fg_cols = [f.name for f in air_quality_fg_v2.features]
    for c in fg_cols:
        if c not in today_row.columns:
            today_row[c] = None
    today_row = today_row[fg_cols]

    air_quality_fg_v2.insert(today_row, wait=True)
    print(f"[air_quality v2] inserted {today_ts.date()} with lag1/2/3.")
else:
    print("[air_quality v2] Not enough strictly previous-day history to compute all three lags. Skipped.")

# ===== End Fix =====



Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.41s) 
2025-11-05 11:48:15,929 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1271980/fs/1265734/fg/1596002


Uploading Dataframe: 100.00% |█████████████████████████████████| Rows 1/1 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: air_quality_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1271980/jobs/named/air_quality_2_offline_fg_materialization/executions
2025-11-05 11:48:35,865 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-05 11:48:39,178 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-05 11:50:08,377 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-05 11:50:08,659 INFO: Waiting for log aggregation to finish.
2025-11-05 11:50:21,228 INFO: Execution finished successfully.
[air_quality v2] inserted 2025-11-05 with lag1/2/3.


In [20]:
air_quality_fg_v2.read().sort_values("date", ascending=False).head(10)   # 看最近 10 天


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.57s) 


,date,pm25,country,city,street,url,pm25_lag1,pm25_lag2,pm25_lag3
4239,2025-11-05 00:00:00+00:00,192.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,157.0,93.0,45.0
2272,2025-11-03 00:00:00+00:00,93.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,45.0,115.0,95.0
2343,2025-11-02 00:00:00+00:00,45.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,115.0,95.0,178.0
257,2025-11-01 00:00:00+00:00,115.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,95.0,178.0,198.0
1247,2025-10-31 00:00:00+00:00,95.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,178.0,198.0,144.0
3609,2025-10-30 00:00:00+00:00,178.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,198.0,144.0,65.0
2221,2025-10-29 00:00:00+00:00,198.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,144.0,65.0,67.0
3483,2025-10-28 00:00:00+00:00,144.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,65.0,67.0,176.0
2107,2025-10-27 00:00:00+00:00,65.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,67.0,176.0,161.0
3225,2025-10-26 00:00:00+00:00,67.0,china,beijing,dongchengdongsi,https://api.waqi.info/feed/@446,176.0,161.0,138.0


In [21]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-05 11:55:16,508 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1271980/fs/1265734/fg/1595945


Uploading Dataframe: 100.00% |█████████████████████████████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1271980/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-05 11:55:36,368 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-05 11:55:39,768 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-05 11:57:07,413 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-05 11:57:07,737 INFO: Waiting for log aggregation to finish.
2025-11-05 11:57:16,860 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 733212
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-05T03:55:16.000507Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
